<a id="0"></a> <br>
### Indice 
- [Librerias](#1)
- [Web scrapping PSN](#2)
    * [Columnas de DF para web scrap](#3)
    * [Primera aproximación al problema](#4)
- [Código web scrappeo sin necesidad de navegador abierto](#5)
- [Funciones y clases](#6)
- [Código principal ](#7)
- [Inicialización de base de datos](#8)
- [Limpieza df](#9)


<a id="1"></a> <br>
### Librerias

In [1]:
import re # Expresiones regulares 
import time
import variables as v
import random
import requests
import pandas as pd

import matplotlib.pyplot as plt # Para visualizaciones en Jupyter no importar script python
import seaborn as sns # Para visualizaciones en Jupyter no importar script python

from datetime import datetime
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC




[Volver al índice](#0)

<a id="3"></a> <br>
## Webscrappeo Psn

##### Queremos hacer una comparativa en el tiempo de los precios de los Juegos de la página web de PSN. 
##### Para ello primero necesitamos scrapear de manera correcta todos los datos para poder almacenarlos y acceder a ellos.

#### Hacemos unas primeras aproximaciones combinando Selenium con Beautiful soup y accediendo a pocas variables.

In [61]:
# Empezamos haciendo un df de que queremos como primera aproximación
df_juegos = pd.DataFrame(columns=["Titulo","Plataforma","Genero","Compañia",
                                "Lanzamiento","Idiomas","Calificación PSN","Número de calificaciones","Calificación 5 estrellas",
                                "Calificación 4 estrellas","Calificación 3 estrellas","Calificación 2 estrellas",
                                "Calificación 1 estrella","Precio_original_con_PSN","Precio_actual_sin_PSN"])

In [63]:
df_juegos.columns

Index(['Titulo', 'Plataforma', 'Genero', 'Compañia', 'Lanzamiento', 'Idiomas',
       'Calificación PSN', 'Número de calificaciones',
       'Calificación 5 estrellas', 'Calificación 4 estrellas',
       'Calificación 3 estrellas', 'Calificación 2 estrellas',
       'Calificación 1 estrella', 'Precio_original_con_PSN',
       'Precio_actual_sin_PSN'],
      dtype='object')

<a id="4"></a> <br>
Columnas de DF para webscrappear

Primero lo que necesitamos es llegar hacia la página web de cada juego concreto, para ello utilizamos selenium

Lo que estamos haciendo con todo este codigo es entrar en la web, ordenarlos por letra de manera ascendente y entrar en la pagina del primer juego para obtener el soup y así mostrar como entramos a cada dato

In [65]:
service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
options = webdriver.ChromeOptions()

driver = webdriver.Chrome(service=service, options=options)

driver.get("https://store.playstation.com/")


lista_tiempo = [1.0,1.5,2.0]

#rechazamos cookies
time.sleep(random.choice(lista_tiempo))
rechazar_cookies = driver.find_element(By.XPATH, '/html/body/div[5]/div[2]/div/div/div[1]/div/div[2]/button/img')
rechazar_cookies.click()

#vamos a la pestaña de explora
time.sleep(random.choice(lista_tiempo))
explora = driver.find_element(By.XPATH, '/html/body/div[3]/section/div/div/div/ul/li[5]/a')
explora.click()


#plat
time.sleep(random.choice(lista_tiempo))
sort_plat = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[4]/button')
sort_plat.click()


#ordenamos
time.sleep(random.choice(lista_tiempo))
sort_gam = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/header/button')
sort_gam.click()

#AZ
time.sleep(random.choice(lista_tiempo))
sort_az = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/div/span/label[3]')
sort_az.click()

time.sleep(random.choice(lista_tiempo))

page_source_init = driver.page_source
soup = bs(page_source_init, 'lxml')
game = 1
select_game = driver.find_element(By.XPATH, f'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/ul/li[{game}]/div/a')
select_game.click()

time.sleep(random.choice(lista_tiempo))

# Info que nos descargamos de la pagina del juego.
page_source = driver.page_source
soup = bs(page_source, 'lxml')
print("Obtenemos nuestra \"sopa\" correspondiente al primer juego")
driver.quit()

Obtenemos nuestra "sopa" correspondiente al primer juego


In [67]:
# Titulo
try:
    print(soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-8 psw-l-line-break-word").get_text())
except:
    print(soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-7 psw-l-line-break-word").get_text())

0 Degrees


In [ ]:
# Precio Original sin PSN en caso de hubiera descuento, en caso de que no exista, error.
soup.find("span",class_="psw-t-title-s psw-c-t-2 psw-t-strike").get_text()


In [69]:
#Precio Actual sin PSN
try:
    print(soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text())
except: 
    print(soup.find("span",class_="psw-t-title-m").get_text())

4,99 €


In [ ]:
# Precio original con PSN, precio anterior sin descuento con PSN , en caso de que no exista, error.
try:
    print(soup.find("span",attrs={'class':'psw-truncate-text-1 psw-p-t-1 psw-l-exclude@desktop'}).get_text())
except: 
    soup.find("span",class_="psw-t-title-s psw-c-t-2 psw-t-strike").get_text()

In [74]:
#Precio Actual con PSN
try:
    print(soup.find("span",attrs={'data-qa':'mfeCtaMain#offer1#finalPrice','class':'psw-t-title-m psw-m-r-4'}).get_text())
except: 
    try:
        print(soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text())
    except: 
        print(soup.find("span",class_="psw-t-title-m").get_text())

4,99 €


In [75]:
# Plataforma
soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#platform-value'}).get_text()


'PS5'

In [76]:
# Genero
soup.find("span", attrs={'style':'text-transform: capitalize;'}).get_text()

'Acción, Puzzle'

In [77]:
# Compañia
soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#publisher-value'}).get_text()

'eastasiasoft'

In [78]:
# Lanzamiento
soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#releaseDate-value'}).get_text()

'31/12/2021'

In [79]:
# Idiomas
soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#subtitles-value'}).get_text()


'Inglés'

In [80]:
# Calificación PSN
soup.find("div", attrs={'class':'psw-t-subtitle psw-t-bold psw-l-line-center','data-qa':'mfe-game-title#average-rating'}).get_text()


'3.86'

In [81]:
# Nº Calificaciones
soup.find("span", attrs={'class':'psw-c-t-2 psw-t-secondary','data-qa':'mfe-star-rating#overall-rating#total-ratings'}).get_text()

'91 calificaciones'

In [82]:
# % 5 estrellas 
soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress5#percentage-label'}).get_text()

'51 %'

In [83]:
# % 4 estrellas 
soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress4#percentage-label'}).get_text()

'10 %'

In [85]:
# % 3 estrellas 
soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress3#percentage-label'}).get_text()

'24 %'

In [86]:
# % 2 estrellas 
soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress2#percentage-label'}).get_text()

'5 %'

In [87]:
# % 1 estrella
soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress1#percentage-label'}).get_text()

'10 %'

In [88]:
# Precio al día
fecha_webs = datetime.now()
fecha_webs = datetime.isoformat(fecha_webs)

Esto fue una primera aproximación a los valores que queremos obtener en nuestro web scrapping.

[Volver al índice](#0)

<a id="4"></a> <br>
### Primera aproximación funcional

In [89]:
### PRUEBA FUNCIONAL WEBSCRAPING DE X JUEGOS ### Completamente funcional, pero mucho tiempo utilizado por campo...

service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
options = webdriver.ChromeOptions()

df_juegos = pd.DataFrame(columns=["Titulo","Plataforma","Genero","Compañia",
                                "Lanzamiento","Idiomas","Calificación PSN","Número de calificaciones","Calificación 5 estrellas",
                                "Calificación 4 estrellas","Calificación 3 estrellas","Calificación 2 estrellas",
                                "Calificación 1 estrella","Precio_original_con_PSN","Precio_actual_sin_PSN"])
driver = webdriver.Chrome(service=service, options=options)

driver.get("https://store.playstation.com/")


lista_tiempo = [1.0,1.5,2.0] 

#rechazamos cookies
time.sleep(random.choice(lista_tiempo))
rechazar_cookies = driver.find_element(By.XPATH, '/html/body/div[5]/div[2]/div/div/div[1]/div/div[2]/button/img')
rechazar_cookies.click()

#vamos a la pestaña de explora
time.sleep(random.choice(lista_tiempo))
explora = driver.find_element(By.XPATH, '/html/body/div[3]/section/div/div/div/ul/li[5]/a')
explora.click()


#plat
time.sleep(random.choice(lista_tiempo))
sort_plat = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[4]/button')
sort_plat.click()


#ordenamos
time.sleep(random.choice(lista_tiempo))
sort_gam = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/header/button')
sort_gam.click()

#AZ
time.sleep(random.choice(lista_tiempo))
sort_az = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/div/span/label[3]')
sort_az.click()

# Obtenemos número de juegos para scrapear
time.sleep(random.choice(lista_tiempo))
page_source_init = driver.page_source
soup = bs(page_source_init, 'lxml')
# Mostrar un numero de juegos limitado
numero_juegos = 10
# numero_juegos = soup.find('div', class_ ='psw-t-body psw-c-t-2').get_text() #Si son todos los de la web

#seleccion de juego

cont = 0
while cont <= 24:
    cont += 1
    time.sleep(random.choice(lista_tiempo))
    select_game = driver.find_element(By.XPATH, f'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/ul/li[{cont}]/div/a')
    select_game.click()

    time.sleep(random.choice(lista_tiempo))

    # Info que nos descargamos de la pagina del juego.
    page_source = driver.page_source
    soup = bs(page_source, 'lxml')

    # Variable por juego
    
    # Titulo
    try:
        titulo = soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-8 psw-l-line-break-word").get_text()
    except:
        titulo = soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-7 psw-l-line-break-word").get_text()
    
    # Precio original
    try:
        precio_original_sn_psn = soup.find("span",class_="psw-t-title-s psw-c-t-2 psw-t-strike").get_text() 
    except:
        precio_original_sn_psn = soup.find("span",class_="psw-t-title-m").get_text()
    
    # Precio actual 
    try:
        precio_actual_sn_psn = soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text()
    except:
        precio_actual_sn_psn = precio_original_sn_psn
    
    # Plataforma
    try:
        plataforma = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#platform-value'}).get_text()
    except:
        plataforma = "No hay información"
    
    # Genero
    try:
        genero = soup.find("span", attrs={'style':'text-transform: capitalize;'}).get_text()
    except:
        genero = "No hay información"
    
    # Compañia
    try:
        compania = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#publisher-value'}).get_text()
    except:
        compania = "No hay información"
    
    # Lanzamiento
    try:  
        lanzamiento = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#releaseDate-value'}).get_text()
    except:
        lanzamiento = "No hay información"
    
    # Idiomas
    try:
        idiomas = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#subtitles-value'}).get_text()
    except:
        idiomas = "No hay información"
    # Nº Calificaciones
    try:
        num_calificaciones = soup.find("span", attrs={'class':'psw-c-t-2 psw-t-secondary','data-qa':'mfe-star-rating#overall-rating#total-ratings'}).get_text()
    except:
        num_calificaciones = "No hay información"
    # Calificación PSN
    
    try:
        calificacion = soup.find("div", attrs={'class':'psw-t-subtitle psw-t-bold psw-l-line-center','data-qa':'mfe-game-title#average-rating'}).get_text()
    except:
        calificacion = "No hay información"
    try:
        calificacion_1 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress1#percentage-label'}).get_text()
    except:
        calificacion_1 = "No hay información"
    try:
        calificacion_2 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress2#percentage-label'}).get_text()
    except:
        calificacion_2 = "No hay información"
    try:
        calificacion_3 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress3#percentage-label'}).get_text()
    except:
        calificacion_3 = "No hay información"
    try:
        calificacion_4 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress4#percentage-label'}).get_text()
    except:
        calificacion_4 = "No hay información"
    try:
        calificacion_5 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress5#percentage-label'}).get_text()
    except:
        calificacion_5 = "No hay información"
    
    # precio_con_mayor_rebaja = "No hay información"
    
    df_juegos.loc[len(df_juegos)] = {"Titulo":titulo,"Plataforma":plataforma,"Genero":genero,"Compañia":compania,"Lanzamiento":lanzamiento,
                                    "Idiomas":idiomas,"Calificación PSN":calificacion,"Número de calificaciones":num_calificaciones,
                                    "Calificación 5 estrellas":calificacion_5,
                                    "Calificación 4 estrellas":calificacion_4,"Calificación 3 estrellas":calificacion_3,
                                    "Calificación 2 estrellas":calificacion_2,"Calificación 1 estrella":calificacion_1,
                                    "Precio original sin PSN":precio_original_sn_psn,"Precio_actual sin PSN":precio_actual_sn_psn}
                                    # "Precio con mayor rebaja":precio_con_mayor_rebaja 
    # time.sleep(random.choice(lista_tiempo))

    driver.back();

    time.sleep(random.choice(lista_tiempo))
    #Comprobar error aquí
    if cont == 24:
        next_page = driver.find_element(By.XPATH,'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/div/nav/button[2]')  
        next_page.click()      
        cont = 0
    elif numero_juegos == len(df_juegos):
        break
    else:
        continue           
driver.quit()


In [91]:
df_juegos.head()

,Titulo,Plataforma,Genero,Compañia,Lanzamiento,Idiomas,Calificación PSN,Número de calificaciones,Calificación 5 estrellas,Calificación 4 estrellas,Calificación 3 estrellas,Calificación 2 estrellas,Calificación 1 estrella,Precio_original_con_PSN,Precio_actual_sin_PSN
0,0 Degrees,PS5,"Acción, Puzzle",eastasiasoft,31/12/2021,Inglés,3.86,91 calificaciones,51 %,10 %,24 %,5 %,10 %,NaN,NaN
1,1001 Spikes,PS4,"Acción, Acción, Arcade",Nicalis Inc.,7/10/2015,Inglés,4.43,893 calificaciones,68 %,20 %,6 %,3 %,4 %,NaN,NaN
2,100ft Robot Golf,PS4,"Grupo, Deporte",NO GOBLIN LLC,18/9/2017,Inglés,3.42,288 calificaciones,38 %,17 %,14 %,12 %,20 %,NaN,NaN
3,101 Ways to Die,PS4,Puzzle,VISION GAMES PUBLISHING,22/3/2016,"Alemán, Español, Francés (Francia), Inglés, It...",3.89,257 calificaciones,46 %,25 %,14 %,4 %,12 %,NaN,NaN
4,10 Second Ninja X,PS4,Puzzle,Curve Digital,19/7/2016,"Alemán, Español, Francés (Francia), Inglés, Ruso",3.52,3.622 calificaciones,47 %,11 %,13 %,6 %,23 %,NaN,NaN


1 minuto aproximadamente 10 juegos, no es un tiempo muy práctico pero recoge los datos de manera correcta.

[Volver al índice](#0)

<a id="5"></a> <br>
Código webscrappeo sin necesidad de navegador abierto

In [ ]:
### PRUEBA FUNCIONAL WEBSCRAPING DE X JUEGOS Y TIEMPO UTILIZADO ### Completamente funcional, pocos campos, meter más cosas genero etc...
# Reset df
service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")


df_juegos = pd.DataFrame(columns=["Titulo","Día y hora","Plataforma","Genero","Compañia",
                                "Lanzamiento","Idiomas","Calificación PSN","Número de calificaciones","Calificación 5 estrellas",
                                "Calificación 4 estrellas","Calificación 3 estrellas","Calificación 2 estrellas",
                                "Calificación 1 estrella","Precio original sin PSN","Precio actual sin PSN","Precio original con PSN","Precio actual con PSN"]) #,"Precio con mayor rebaja"

driver = webdriver.Chrome(service=service, options=options)


driver.get("https://store.playstation.com/")


lista_tiempo = [1.9,2.0,2.1,2.2]

#rechazamos cookies
time.sleep(random.choice(lista_tiempo))
rechazar_cookies = driver.find_element(By.XPATH, '/html/body/div[5]/div[2]/div/div/div[1]/div/div[2]/button/img')
rechazar_cookies.click()

#vamos a la pestaña de explora
time.sleep(random.choice(lista_tiempo))
explora = driver.find_element(By.XPATH, '/html/body/div[3]/section/div/div/div/ul/li[5]/a')
explora.click()


#plat
time.sleep(random.choice(lista_tiempo))
sort_plat = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[4]/button')
sort_plat.click()


#ordenamos
time.sleep(random.choice(lista_tiempo))
sort_gam = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/header/button')
sort_gam.click()

#AZ
time.sleep(random.choice(lista_tiempo))
sort_az = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/div/span/label[3]')
sort_az.click()

# Obtenemos número de juegos para scrapear
time.sleep(random.choice(lista_tiempo))
page_source = driver.page_source
soup = bs(page_source, 'lxml')
# Mostrar un numero de juegos limitado
numero_juegos = 1000
# numero_juegos = soup.find('div', class_ ='psw-t-body psw-c-t-2').get_text() #Si son todos los de la web

#seleccion de juego

cont = 0
while cont <= 24:
    cont += 1
    time.sleep(random.choice(lista_tiempo))
    select_game = driver.find_element(By.XPATH, f'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/ul/li[{cont}]/div/a')
    select_game.click()

    time.sleep(random.choice(lista_tiempo))

    # Info que nos descargamos de la pagina del juego.
    page_source = driver.page_source
    soup = bs(page_source, 'html')

    # Variable por juego
    
    # Titulo
    try:
        titulo = soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-8 psw-l-line-break-word").get_text()
    except:
        titulo = soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-7 psw-l-line-break-word").get_text()
    
    # Día y hora de webscrappeo
    
        fecha_webs = datetime.now()
        fecha_webs = datetime.isoformat(fecha_webs)
        
    # Precio original sin PSN
    try:
        precio_original_sn_psn = soup.find("span",class_="psw-t-title-s psw-c-t-2 psw-t-strike").get_text() 
    except:
        precio_original_sn_psn = soup.find("span",class_="psw-t-title-m").get_text()
        
    # Precio original con PSN
    try:
        precio_original_cn_psn = soup.find("span",attrs={'data-qa':'mfeCtaMain#offer1#originalPrice','class':'psw-t-title-s psw-c-t-2 psw-t-strike'}).get_text()
    except: 
        try:
            precio_original_cn_psn = soup.find("span",attrs={'class':'psw-truncate-text-1 psw-p-t-1 psw-l-exclude@desktop'}).get_text()
        except:
            precio_original_cn_psn = precio_original_sn_psn
    # Precio actual sin PSN
    try:
        precio_actual_sn_psn = soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text()
    except:
        precio_actual_sn_psn = precio_original_sn_psn
    
    #Precio Actual con PSN
    try:
        precio_actual_cn_psn = soup.find("span",attrs={'data-qa':'mfeCtaMain#offer1#finalPrice','class':'psw-t-title-m psw-m-r-4'}).get_text()
    except: 
        try:
            precio_actual_cn_psn = soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text()
        except: 
            precio_actual_cn_psn = precio_actual_sn_psn
    
    # Plataforma
    try:
        plataforma = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#platform-value'}).get_text()
    except:
        plataforma = "No hay información"
    
    # Genero
    try:
        genero = soup.find("span", attrs={'style':'text-transform: capitalize;'}).get_text()
    except:
        genero = "No hay información"
    
    # Compañia
    try:
        compania = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#publisher-value'}).get_text()
    except:
        compania = "No hay información"
    
    # Lanzamiento
    try:  
        lanzamiento = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#releaseDate-value'}).get_text()
    except:
        lanzamiento = "No hay información"
    
    # Idiomas
    try:
        idiomas = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#subtitles-value'}).get_text()
    except:
        idiomas = "No hay información"
    # Nº Calificaciones
    try:
        num_calificaciones = soup.find("span", attrs={'class':'psw-c-t-2 psw-t-secondary','data-qa':'mfe-star-rating#overall-rating#total-ratings'}).get_text()
    except:
        num_calificaciones = "No hay información"
    # Calificación PSN
    
    try:
        calificacion = soup.find("div", attrs={'class':'psw-t-subtitle psw-t-bold psw-l-line-center','data-qa':'mfe-game-title#average-rating'}).get_text()
    except:
        calificacion = "No hay información"
    try:
        calificacion_1 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress1#percentage-label'}).get_text()
    except:
        calificacion_1 = "No hay información"
    try:
        calificacion_2 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress2#percentage-label'}).get_text()
    except:
        calificacion_2 = "No hay información"
    try:
        calificacion_3 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress3#percentage-label'}).get_text()
    except:
        calificacion_3 = "No hay información"
    try:
        calificacion_4 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress4#percentage-label'}).get_text()
    except:
        calificacion_4 = "No hay información"
    try:
        calificacion_5 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress5#percentage-label'}).get_text()
    except:
        calificacion_5 = "No hay información"
    
    # precio_con_mayor_rebaja = "No hay información"
    
    df_juegos.loc[len(df_juegos)] = {"Titulo":titulo,"Día y hora":fecha_webs,"Plataforma":plataforma,"Genero":genero,"Compañia":compania,"Lanzamiento":lanzamiento,
                                    "Idiomas":idiomas,"Calificación PSN":calificacion,"Número de calificaciones":num_calificaciones,
                                    "Calificación 5 estrellas":calificacion_5,
                                    "Calificación 4 estrellas":calificacion_4,"Calificación 3 estrellas":calificacion_3,
                                    "Calificación 2 estrellas":calificacion_2,"Calificación 1 estrella":calificacion_1,
                                    "Precio original sin PSN":precio_original_sn_psn,"Precio actual sin PSN":precio_actual_sn_psn,"Precio original con PSN":precio_original_cn_psn, "Precio actual con PSN":precio_actual_cn_psn}
                                    # "Precio con mayor rebaja":precio_con_mayor_rebaja 
    # time.sleep(random.choice(lista_tiempo))

    driver.back();

    time.sleep(random.choice(lista_tiempo))
    #Comprobar error aquí
    if cont == 24:
        next_page = driver.find_element(By.XPATH,'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/div/nav/button[2]')  
        next_page.click()
        time.sleep(random.choice(lista_tiempo))      
        cont = 0
    elif numero_juegos == len(df_juegos):
        break
    else:
        continue           
driver.quit()

Una vez hecho esto intentamos alojar funciones y clases para poder hacerlo todavía más 

<a id="6"></a> <br>
### Funciones y Clases 

#### Funciones

In [2]:
def carga_pagina_inicial():
    '''
    En esta función estamos haciendo la carga de la página inicial donde más adelante,
    empezaremos a recoger información sobre todos los juegos de la plataforma
    
    '''
    from selenium.common.exceptions import TimeoutException
    from fake_useragent import UserAgent
    "fake_user_agent"
    ua = UserAgent()
    timeout = 10
    
    # lista_tiempo = [3,3.1,3.2]
    #rechazamos cookies

    try:
        butt_coo = EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div[2]/div/div/div[1]/div/div[2]/button/img'))
        WebDriverWait(driver, timeout).until(butt_coo)
    except TimeoutException:
        print("Timed out waiting for sort button to appear")

    rechazar_cookies = driver.find_element(By.XPATH, '/html/body/div[5]/div[2]/div/div/div[1]/div/div[2]/button/img')
    rechazar_cookies.click()

    #vamos a la pestaña de explora

    try:
        expl_butt = EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/section/div/div/div/ul/li[5]/a'))
        WebDriverWait(driver, timeout).until(expl_butt)
    except TimeoutException:
        print("Timed out waiting for sort button to appear")

    explora = driver.find_element(By.XPATH, '/html/body/div[3]/section/div/div/div/ul/li[5]/a')
    explora.click()

    #return

    # Si queremos ordenar en ascendente 
    # plat
    
    try:
        sort_butt = EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[4]/button'))
        WebDriverWait(driver, timeout).until(sort_butt)
    except TimeoutException:
        print("Timed out waiting for sort button to appear")


    sort_plat = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[4]/button')
    sort_plat.click()


    #ordenamos

    try:
        sort_g = EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/header/button'))
        WebDriverWait(driver, timeout).until(sort_g)
    except TimeoutException:
        print("Timed out waiting for sort button to appear")


    sort_gam = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/header/button')
    sort_gam.click()

    #AZ


    try:
        sort_az_ = EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/div/span/label[3]'))
        WebDriverWait(driver, timeout).until(sort_az_)
    except TimeoutException:
        print("Timed out waiting for sort button to appear")

    sort_az = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/div/span/label[3]')
    sort_az.click()
   

    driver.implicitly_wait(10)

    return
    
            
def pagina_concreta_carga(pagina):
    pag = 1
    while pag != pagina:
        next_page = driver.find_element(By.XPATH,'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/div/nav/button[2]')  
        next_page.click()
        pag += 1
        
def id_juego(ref):
    
    id_juego = re.findall(r"\d+",ref)
    id_juego = int(id_juego[0])
    return id_juego

#### Clases


In [3]:
class info_game:
    
    """
    Una clase hecha para tener toda la información de cada juego concreto
    
    """
    def __init__(self,soup,etiqueta,atributo):
        """Constructor donde defino la sopa de BS que necesito, la etiqueta y el atributo o atributos concretos

        Args:
            soup (list): La sopa de xml que recibo de la web
            etiqueta (string): la etiqueta que busco en cada scrapeo
            atributo (list): lista de diccionarios que necesito para obtener cada detalle de cada juego
        """
        self.sopa = soup
        self.atribs = atributo
        self.etiq = etiqueta
    
    def caracteristica_tipo(self,mensaje_concreto_error_df,cal=False):
        """ 
        Metodo que nos devuelve la caracteristica concreta que queramos para cada juego
        
        Returns:
            caracarct(str): Devuelve la caracteristica concreta de cada tipo
        """
        if cal == True:
            calificacion_list = []
            for i in range(0,5):   
                calificacion_list.append(self.sopa.find(self.etiq, attrs=self.atribs[i]).get_text())
            return calificacion_list
        else:
                      
            for indice, valor in enumerate(self.atribs):
                
                try:
                    caracarct = self.sopa.find(self.etiq, attrs=self.atribs[indice]).get_text()
                    break
                except AttributeError:
                    caracarct = mensaje_concreto_error_df        
        
            return caracarct    

##### Prueba funcional con Funciones y Clases sobre un Juego

In [12]:
# Codigo para prueba de clases que me suelte soup y pruebe todas las posibilidades
# Prueba numero de juegos
from fake_useragent import UserAgent
from selenium.common.exceptions import TimeoutException
"fake_user_agent"
ua = UserAgent()
service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
timeout = 10

df_juegos = pd.DataFrame(columns=["Titulo","Día y hora","Plataforma","Genero","Compañia",
                                "Lanzamiento","Idiomas","Calificación PSN","Número de calificaciones","Calificación 5 estrellas",
                                "Calificación 4 estrellas","Calificación 3 estrellas","Calificación 2 estrellas",
                                "Calificación 1 estrella","Precio original sin PSN","Precio actual sin PSN","Precio original con PSN","Precio actual con PSN"]) #,"Precio con mayor rebaja"

driver = webdriver.Chrome(service=service, options=options)

link_inicial = "https://store.playstation.com/"

driver.get(link_inicial)

page = 1
cont = 15
lista_tiempo = [3,3.1,3.2]

carga_pagina_inicial()

try:
    sel_game = EC.presence_of_element_located((By.XPATH, f'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/ul/li[{cont+1}]/div/a'))
    WebDriverWait(driver, timeout).until(sel_game)
except TimeoutException:
    print(f"Timed out waiting for game to appear, game number {cont}")
url = driver.current_url  
headers = {'User-Agent': ua.random}
response = requests.get(url, headers=headers)
soup_pagina_entera = bs(response.text,features="lxml")
url_game = soup_pagina_entera.select_one(f'[data-qa="ems-sdk-grid#productTile{cont}"] a')
href_valor = url_game.get('href')
link_juego = link_inicial + href_valor
# obtenemos info del juego         
headers = {'User-Agent': ua.random}
response = requests.get(link_juego, headers=headers)
soup = bs(response.text,features="lxml")

time.sleep(random.choice(lista_tiempo))

# Info que nos descargamos de la pagina del juego.

#Titulo e id del juego 
title_info = info_game(soup,"h1",[{"class":"psw-m-b-5 psw-t-title-l psw-t-size-6 psw-l-line-break-word"},
                              {"class":"psw-m-b-5 psw-t-title-l psw-t-size-7 psw-l-line-break-word"},
                              {"class":"psw-m-b-5 psw-t-title-l psw-t-size-8 psw-l-line-break-word"}])

id_juego = re.findall(r"\d+",href_valor)
id_juego = int(id_juego[0])
print(id_juego)

titulo = title_info.caracteristica_tipo("Error al recoger el título")
print(titulo)

# Día y hora de webscrappeo
fecha_webs = datetime.now()
fecha_webs = datetime.isoformat(fecha_webs)
print(fecha_webs)

# Precio original sin PSN
org_price_without_psn = info_game(soup,"span",[{"class":"psw-t-title-s psw-c-t-2 psw-t-strike"},
                                            {"class":"psw-t-title-m"}])
precio_original_sn_psn = org_price_without_psn.caracteristica_tipo("Error al recoger precio original sin psn")
print(precio_original_sn_psn)

# Precio original con PSN --> No aparece , solo null
org_price_with_psn = info_game(soup,"span",[{'data-qa':'mfeCtaMain#offer1#originalPrice','class':'psw-t-title-s psw-c-t-2 psw-t-strike'},
                                            {"class":"psw-t-title-s psw-c-t-2 psw-t-strike"},{"class":"psw-t-title-m"}])
precio_original_cn_psn = org_price_with_psn.caracteristica_tipo("Error al recoger precio original con psn")
print("Precio original con PSN",precio_original_cn_psn)

# Precio actual sin PSN
act_price_without_psn = info_game(soup,"span",[{'class':"psw-t-title-m psw-m-r-4"},{"class":"psw-t-title-m"}])
precio_actual_sn_psn = act_price_without_psn.caracteristica_tipo("Error al recoger precio actual sin psn")
print("Precio actual sin PSN", precio_actual_sn_psn)

# Precio actual con PSN
act_price_with_psn = info_game(soup,"span",[{'data-qa':'mfeCtaMain#offer1#finalPrice','class':'psw-t-title-m psw-m-r-4'},
                                            {'class':"psw-t-title-m psw-m-r-4"},{"class":"psw-t-title-m"}])
precio_actual_cn_psn = act_price_with_psn.caracteristica_tipo("Error al recoger precio original con psn")
print("Precio actual con PSN",precio_actual_cn_psn)

#Plataforma
pltform = info_game(soup,"dd",[{'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max',
                               'data-qa':'gameInfo#releaseInformation#platform-value'}])

plataforma = pltform.caracteristica_tipo("Error al recoger plataforma")
print(plataforma)

# Genero
gnr = info_game(soup,"dd",[{'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max',
                            'data-qa':'gameInfo#releaseInformation#genre-value'}])

genero = gnr.caracteristica_tipo("Error al recoger género")
print(genero)

# Compañia
company = info_game(soup,"dd",[{'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max',
                                'data-qa':'gameInfo#releaseInformation#publisher-value'}])

compania = company.caracteristica_tipo("Error al recoger la compañia")
print(compania)

# Lanzamiento
lanz = info_game(soup,"dd",[{'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max',
                             'data-qa':'gameInfo#releaseInformation#releaseDate-value'}])

lanzamiento = lanz.caracteristica_tipo("Error al recoger Lanzamiento")
print(lanzamiento)

# Idiomas
lng = info_game(soup,"dd",[{'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max',
                            'data-qa':'gameInfo#releaseInformation#subtitles-value'}])

idiomas = lng.caracteristica_tipo("Error al recoger Idiomas")
print(idiomas)

# Nº de calificaciones

num_cal = info_game(soup,"span",[{'class':'psw-c-t-2 psw-t-secondary',
                                 'data-qa':'mfe-star-rating#overall-rating#total-ratings'}])

num_calificaciones = num_cal.caracteristica_tipo("Error al recoger num calificaciones")
print(num_calificaciones)

# Calificación PSN
cal_psn = info_game(soup,"div",[{'class':'psw-t-subtitle psw-t-bold psw-l-line-center','data-qa':'mfe-game-title#average-rating'}])

calificacion = cal_psn.caracteristica_tipo("Error al recoger Calificación PSN")
print(calificacion)


# Calificaciones por estrellas

cal_stars = info_game(soup,"span",[{'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress1#percentage-label'},
                             {'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress2#percentage-label'},
                             {'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress3#percentage-label'},
                             {'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress4#percentage-label'},
                             {'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress5#percentage-label'}])

list_stars = cal_stars.caracteristica_tipo("Error al obtener calificaciones por estrella",cal=True)
calificacion_1 = list_stars[0]
calificacion_2 = list_stars[1]
calificacion_3 = list_stars[2]
calificacion_4 = list_stars[3]
calificacion_5 = list_stars[4]
print(calificacion_1,calificacion_2,calificacion_3,calificacion_4,calificacion_5)

print(href_valor)


10002456
Marvel’s Spider-Man 2
2024-03-10T22:52:33.990287
79,99 €
Precio original con PSN 79,99 €
Precio actual sin PSN 79,99 €
Precio actual con PSN 79,99 €
PS5
Acción
Sony Interactive Entertainment Europe
19/10/2023
Alemán, Checo, Danés, Español, Finlandés, Francés (Francia), Griego, Húngaro, Inglés, Italiano, Neerlandés, Noruego, Polaco, Portugués (Portugal), Ruso, Sueco, Árabe
109.083 calificaciones
4.88
1 % 1 % 1 % 4 % 93 %
/es-es/concept/10002456


In [13]:
href_store_us = href_valor
href_store_us.replace('/es-es','/en-us')

'/en-us/concept/10002456'

#### En el main.py hay algunos errores en algunos juegos, por lo que intento hacer una segunda comprobación antes de empezar a extraer datos de una sopa, para ello recogemos el id del producto y lo comparamos con el id recogido del URL

In [ ]:
soup.find_all('a')

In [ ]:
# Primer acercamiento a doble check info
dict_comprobacion_id = soup.find('a', class_="psw-link psw-content-link").get_attribute_list("data-telemetry-meta")[0]

In [41]:
# Segunda aproximación
dict_comprobacion_id = soup.find('a', attrs={'data-track-click':"starRating:selectStarRatingLink"}).get_attribute_list("data-telemetry-meta")[0]

In [55]:
# Tercera aproximación
dict_comprobacion_id = soup.find("button",attrs={"data-qa":"mfeCtaMain#cta#action"}).get_attribute_list("data-telemetry-meta")[0]

# Nos quedamos con la tercera aproximación que es la menos errores tiene.

In [ ]:
# <a data-qa="" data-track-click="starRating:selectStarRatingLink" data-telemetry-meta="{&quot;conceptId&quot;:&quot;231287&quot;,&quot;productId&quot;:&quot;EP3868-CUSA11045_00-6589215874582693&quot;}" id="" class="psw-link psw-content-link psw-ally-indicator" aria-label="" type="" href="#star-rating" rel="noopener noreferrer"><span class="psw-sr-only">Calificación media de 3.49 estrellas de un total de cinco estrellas en 86 calificaciones</span><div aria-hidden="true" class="psw-l-inline psw-p-y-xs"><div class="psw-t-subtitle psw-t-bold psw-l-line-center" data-qa="mfe-game-title#average-rating">3.49</div><span class="psw-p-x-1 psw-l-line-center psw-l-line-left" data-qa="mfe-game-title#star-rating"><span data-qa="mfe-game-title#star-rating#rating-1#icon" class="psw-icon psw-icon--starrating-full psw-icon psw-icon-size-3 psw-icon--starrating-full"><svg aria-hidden="true" focusable="false"><title></title><use href="#ps-icon:starrating-full"></use></svg></span><span data-qa="mfe-game-title#star-rating#rating-2#icon" class="psw-icon psw-icon--starrating-full psw-icon psw-icon-size-3 psw-icon--starrating-full"><svg aria-hidden="true" focusable="false"><title></title><use href="#ps-icon:starrating-full"></use></svg></span><span data-qa="mfe-game-title#star-rating#rating-3#icon" class="psw-icon psw-icon--starrating-full psw-icon psw-icon-size-3 psw-icon--starrating-full"><svg aria-hidden="true" focusable="false"><title></title><use href="#ps-icon:starrating-full"></use></svg></span><span data-qa="mfe-game-title#star-rating#rating-4#icon" class="psw-icon psw-icon--starrating-half psw-icon psw-icon-size-3 psw-icon--starrating-half"><svg aria-hidden="true" focusable="false"><title></title><use href="#ps-icon:starrating-half"></use></svg></span><span data-qa="mfe-game-title#star-rating#rating-5#icon" class="psw-icon psw-icon--starrating-empty psw-icon psw-icon-size-3 psw-icon--starrating-empty"><svg aria-hidden="true" focusable="false"><title></title><use href="#ps-icon:starrating-empty"></use></svg></span></span><div class="psw-t-subtitle psw-l-line-center" data-qa="mfe-game-title#rating-count">86 calificaciones</div></div></a>

In [56]:
dict_comprobacion_id

'{"conceptId":"10004473","ctaIndex":0,"ctaSubType":"add_to_cart","ctaType":"ADD_TO_CART","interactAction":"click add to cart","interactCta":"add to cart","productId":"EP0102-PPSA07412_00-RE4RMAINGAME0000","productDetail":[{"productId":"EP0102-PPSA07412_00-RE4RMAINGAME0000","productName":"Resident Evil 4 PS4 & PS5","productToSkuIds":["EP0102-PPSA07412_00-RE4RMAINGAME0000-E005"],"productPriceDetail":[{"rewardId":"OFFER-PROD-7518_25","offerCampaignId":"PROMO-PROD-00007518","originalPriceFormatted":"39,99\xa0€","originalPriceValue":3999,"discountPriceFormatted":"29,99\xa0€","discountPriceValue":2999,"priceCurrencyCode":"EUR","offerBranding":"NONE","offerQualification":[],"offerExclusive":false,"offerApplied":true,"offerApplicability":"APPLICABLE","offerIsTiedToSubscription":false}]}],"skuDetail":[{"skuId":"EP0102-PPSA07412_00-RE4RMAINGAME0000-E005","skuPriceDetail":[{"rewardId":"OFFER-PROD-7518_25","offerCampaignId":"PROMO-PROD-00007518","originalPriceFormatted":"39,99\xa0€","originalPrice

In [22]:
import json
conv_json = json.loads(dict_comprobacion_id)
id_juego_real_soup = int(conv_json["conceptId"])
id_juego_real_soup

##### Una vez visto que podemos acceder a toda la info solo por el boton de reserva/compra se me ocurre que podemos obtener informaciónes que dan error desde ahí directamente por si el código fuente cambiara de un día para otro, que en ocasiones ocurre.

##### Tenemos en el main.py algunos problemas con los titulos, buscando en la página web resulta que el boton tanto de comprar como reservar tiene toda la info disponible, asique empiezo a buscar otra forma de hacer el webscrapping con el boton concreto de compra y obtenerlo todo desde allí.

In [19]:
# Titulo
dict_titulo = soup.find('button',attrs={"data-qa" : "mfeCtaMain#cta#action"}).get_attribute_list("data-telemetry-meta")

In [33]:
dict_titulo = soup.find('button',attrs={"data-qa" : "mfeCtaMain#cta#action"}).get_attribute_list("data-telemetry-meta")
json_dict = json.loads(dict_titulo[0])
json_dict['productDetail'][0]['productName']

'Resident Evil 4 PS4 & PS5'

[Volver al índice](#0)

Guardamos ID de cada juego

In [14]:
print(href_valor)

/es-es/concept/10007176


In [18]:
id_juego = re.findall(r"\d+",href_valor)

print(id_juego)

['10007176']


Como obtenemos el numero de juegos

In [64]:
# Prueba numero de juegos
from fake_useragent import UserAgent
"fake_user_agent"
ua = UserAgent()
service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
timeout = 10

df_juegos = pd.DataFrame(columns=["Titulo","Día y hora","Plataforma","Genero","Compañia",
                                "Lanzamiento","Idiomas","Calificación PSN","Número de calificaciones","Calificación 5 estrellas",
                                "Calificación 4 estrellas","Calificación 3 estrellas","Calificación 2 estrellas",
                                "Calificación 1 estrella","Precio original sin PSN","Precio actual sin PSN","Precio original con PSN","Precio actual con PSN"]) #,"Precio con mayor rebaja"

driver = webdriver.Chrome(service=service, options=options)

link_inicial = "https://store.playstation.com/"

driver.get(link_inicial)


lista_tiempo = [3,3.1,3.2]

carga_pagina_inicial()

page_source = driver.page_source
soup = bs(page_source, 'html.parser')
numero_juegos = soup.find('div', class_= "ems-sdk-active-filters psw-m-b-8 psw-m-t-4").get_text()
numero_real = re.findall(r"\d+",numero_juegos)
numero_real = int(numero_real[0])
print(type(numero_real))
print(numero_real)

<class 'int'>
8439


[Volver al índice](#0)

<a id="8"></a> <br>
Creando base de datos en server elephant

In [8]:
import psycopg2

conn = psycopg2.connect(database=v.database,
                        host=v.host,
                        user=v.user,
                        password=v.password,
                        port=v.port)

In [9]:
cursor = conn.cursor()

Creación de tablas

In [10]:
creacion_tablas = '''
-- Tabla Juegos 
CREATE TABLE game_info (
  game_id INTEGER PRIMARY KEY,
  game_name VARCHAR,
  genre_id INTEGER,
  lang_id INTEGER,
  comp_id INTEGER,
  platform_id INTEGER,
  FOREIGN KEY (genre_id) REFERENCES genre(genre_id),
  FOREIGN KEY (lang_id) REFERENCES language(lang_id),
  FOREIGN KEY (comp_id) REFERENCES company(comp_id),
  FOREIGN KEY (platform_id) REFERENCES platform(platform_id)
);
-- Tabla Ratings
CREATE TABLE ratings (
  rating_id INTEGER PRIMARY KEY,
  game_id INTEGER,
  n_ratings_psn INTEGER,
  pro_rating FLOAT,
  cal_5_stars_pct FLOAT,
  cal_4_stars_pct FLOAT,
  cal_3_stars_pct FLOAT,
  cal_2_stars_pct FLOAT,
  cal_1_star_pct FLOAT,
  FOREIGN KEY (game_id) REFERENCES game_info(game_id)
);

-- Tabla precios
CREATE TABLE prices (
  price_id INTEGER PRIMARY KEY,
  game_id INTEGER,
  time FLOAT,
  original_price_without_plus FLOAT,
  original_price_with_plus FLOAT,
  discounted_price_without_plus FLOAT,
  discounted_price_with_plus FLOAT,
  FOREIGN KEY (game_id) REFERENCES game_info(game_id)
);

-- Tabla Plataforma
CREATE TABLE platform (
  platform_id INTEGER PRIMARY KEY,
  platform_name VARCHAR
);

-- Tabla genero
CREATE TABLE genre (
  genre_id INTEGER PRIMARY KEY,
  genre_name VARCHAR
);

-- Tabla lenguaje
CREATE TABLE language (
  lang_id INTEGER PRIMARY KEY,
  lang_name VARCHAR
);

-- Tabla compañia 
CREATE TABLE company (
  comp_id INTEGER PRIMARY KEY,
  comp_name VARCHAR
);

'''

In [ ]:
cursor.execute(creacion_tablas)

In [12]:
conn.commit()
cursor.close()
conn.close()

Prueba de inserts en BBDD

In [75]:
import psycopg2
from sqlalchemy import create_engine

conn = psycopg2.connect(database=v.database,
                        host=v.host,
                        user=v.user,
                        password=v.password,
                        port=v.port)

cursor = conn.cursor()

In [78]:
conn.close()

Query prueba insertar datos

# Banco de pruebas

<a id="9"></a> <br>
Errores listas .py

In [1]:
[221941, 205132, 10001023, 235080, 224832, 202007, 227255, 202523, 229776, 203706, 202665, 208240, 204923, 201386, 234597, 10010335, 216068, 232640, 10007962, 10001257, 10001949, 10008926, 10009430, 233506, 10008712, 10003393, 10007921, 10008377, 10010438, 232136, 201687, 232798, 10005373, 10007580, 10007315, 202006, 222257, 10008416, 10009665, 232230, 10009252, 10003548, 10008403, 10005702, 10000500, 10003237, 202939, 10008737, 10008487, 10009778, 10009509, 229436, 10007261, 10009541, 10010442, 10005760, 232397, 216508, 10009421, 10008557, 10008524, 10009752, 10000976, 10009242, 10004289, 10009679, 10002260, 10002211, 205520, 10008940]

[(0, 18),
 (19, 19),
 (4, 23),
 (8, 23),
 (23, 25),
 (7, 46),
 (1, 54),
 (23, 71),
 (13, 82),
 (20, 82),
 (7, 99),
 (12, 99),
 (5, 104),
 (10, 107),
 (11, 107),
 (0, 123),
 (21, 123),
 (11, 124),
 (13, 127),
 (9, 145),
 (4, 159),
 (2, 170),
 (14, 172),
 (22, 172),
 (19, 173),
 (12, 177),
 (13, 177),
 (19, 179),
 (4, 190),
 (10, 191),
 (7, 197),
 (6, 203),
 (11, 204),
 (16, 210),
 (23, 210),
 (18, 212),
 (10, 215),
 (13, 218),
 (13, 220),
 (16, 220),
 (15, 221),
 (4, 227),
 (19, 230),
 (11, 232),
 (6, 235),
 (1, 236),
 (9, 237),
 (23, 237),
 (11, 240),
 (22, 240),
 (23, 241),
 (20, 242),
 (4, 243),
 (10, 243),
 (19, 244),
 (1, 246),
 (12, 247),
 (20, 269),
 (3, 270),
 (12, 280),
 (14, 292),
 (0, 294),
 (13, 295),
 (15, 309),
 (10, 325),
 (18, 330),
 (14, 332),
 (23, 336),
 (5, 340),
 (9, 345),
 (13, 349),
 (21, 353),
 (19, 355),
 (4, 358)]

#### Estoy intentando obtener los datos de otras stores en este caso a modo de prototipo, las de EEUU y JPN

In [44]:
# Codigo para prueba de clases que me suelte soup y pruebe todas las posibilidades
# Prueba numero de juegos
from fake_useragent import UserAgent
from selenium.common.exceptions import TimeoutException
"fake_user_agent"
ua = UserAgent()
service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
timeout = 10

df_juegos_es = pd.DataFrame(columns=["Titulo","Día y hora","Plataforma","Genero","Compañia",
                                "Lanzamiento","Idiomas","Calificación PSN","Número de calificaciones","Calificación 5 estrellas",
                                "Calificación 4 estrellas","Calificación 3 estrellas","Calificación 2 estrellas",
                                "Calificación 1 estrella","Precio original sin PSN","Precio actual sin PSN","Precio original con PSN","Precio actual con PSN","País Store"]) #,"Precio con mayor rebaja"

#Juegos store nueva, meter en codigo .py
df_juegos_us = df_juegos_es.copy()
df_juegos_jp = df_juegos_es.copy()
df_juegos = [df_juegos_es,df_juegos_us,df_juegos_jp]
lista_store = ["ESP","USA","JAP"]

driver = webdriver.Chrome(service=service, options=options)

link_inicial = "https://store.playstation.com/"

driver.get(link_inicial)

page = 4
cont = 0
lista_tiempo = [3,3.1,3.2]

carga_pagina_inicial()
pagina_concreta_carga(page)

try:
    sel_game = EC.presence_of_element_located((By.XPATH, f'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/ul/li[{cont+1}]/div/a'))
    WebDriverWait(driver, timeout).until(sel_game)
except TimeoutException:
    print(f"Timed out waiting for game to appear, game number {cont}")
url = driver.current_url  
headers = {'User-Agent': ua.random}
response = requests.get(url, headers=headers)
soup_pagina_entera = bs(response.text,features="lxml")
url_game = soup_pagina_entera.select_one(f'[data-qa="ems-sdk-grid#productTile{cont}"] a')

# Obtenemos del href de la store española, pero también podemos obtener info de otras stores 
href_store_es = url_game.get('href')

href_store_us = href_store_es.replace('/es-es','/en-us')
href_store_jp = href_store_es.replace('/es-es','/ja-jp')

href_valor_list = [href_store_es,href_store_us,href_store_jp]

for i in range(len(href_valor_list)):

    pais_store = lista_store[i]
    
    print(href_valor_list[i])
    link_juego = link_inicial + href_valor_list[i]
    # obtenemos info del juego         
    headers = {'User-Agent': ua.random}
    response = requests.get(link_juego, headers=headers)
    soup = bs(response.text,features="lxml")

    time.sleep(random.choice(lista_tiempo))

    # Info que nos descargamos de la pagina del juego.

    #Titulo e id del juego 
    title_info = info_game(soup,"h1",[{"class":"psw-m-b-5 psw-t-title-l psw-t-size-6 psw-l-line-break-word"},
                                {"class":"psw-m-b-5 psw-t-title-l psw-t-size-7 psw-l-line-break-word"},
                                {"class":"psw-m-b-5 psw-t-title-l psw-t-size-8 psw-l-line-break-word"}])

    id_juego = re.findall(r"\d+",href_valor_list[i])
    id_juego = int(id_juego[0])
    print(id_juego)

    titulo = title_info.caracteristica_tipo("Error al recoger el título")
    print(titulo)

    # Día y hora de webscrappeo
    fecha_webs = datetime.now()
    fecha_webs = datetime.isoformat(fecha_webs)
    print(fecha_webs)

    # Precio original sin PSN
    org_price_without_psn = info_game(soup,"span",[{"class":"psw-t-title-s psw-c-t-2 psw-t-strike"},
                                                {"class":"psw-t-title-m"}])
    precio_original_sn_psn = org_price_without_psn.caracteristica_tipo("Error al recoger precio original sin psn")
    print("Precio original sin PSN" ,precio_original_sn_psn)

    # Precio original con PSN --> No aparece , solo null
    org_price_with_psn = info_game(soup,"span",[{'data-qa':'mfeCtaMain#offer1#originalPrice','class':'psw-t-title-s psw-c-t-2 psw-t-strike'},
                                                {"class":"psw-t-title-s psw-c-t-2 psw-t-strike"},{"class":"psw-t-title-m"}])
    precio_original_cn_psn = org_price_with_psn.caracteristica_tipo("Error al recoger precio original con psn")
    print("Precio original con PSN",precio_original_cn_psn)

    # Precio actual sin PSN
    act_price_without_psn = info_game(soup,"span",[{'class':"psw-t-title-m psw-m-r-4"},{"class":"psw-t-title-m"}])
    precio_actual_sn_psn = act_price_without_psn.caracteristica_tipo("Error al recoger precio actual sin psn")
    print("Precio actual sin PSN", precio_actual_sn_psn)

    # Precio actual con PSN
    act_price_with_psn = info_game(soup,"span",[{'data-qa':'mfeCtaMain#offer1#finalPrice','class':'psw-t-title-m psw-m-r-4'},
                                                {'class':"psw-t-title-m psw-m-r-4"},{"class":"psw-t-title-m"}])
    precio_actual_cn_psn = act_price_with_psn.caracteristica_tipo("Error al recoger precio original con psn")
    print("Precio actual con PSN",precio_actual_cn_psn)

    #Plataforma
    pltform = info_game(soup,"dd",[{'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max',
                                'data-qa':'gameInfo#releaseInformation#platform-value'}])

    plataforma = pltform.caracteristica_tipo("Error al recoger plataforma")
    print(plataforma)

    # Genero
    gnr = info_game(soup,"dd",[{'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max',
                                'data-qa':'gameInfo#releaseInformation#genre-value'}])

    genero = gnr.caracteristica_tipo("Error al recoger género")
    print(genero)

    # Compañia
    company = info_game(soup,"dd",[{'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max',
                                    'data-qa':'gameInfo#releaseInformation#publisher-value'}])

    compania = company.caracteristica_tipo("Error al recoger la compañia")
    print(compania)

    # Lanzamiento
    lanz = info_game(soup,"dd",[{'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max',
                                'data-qa':'gameInfo#releaseInformation#releaseDate-value'}])

    lanzamiento = lanz.caracteristica_tipo("Error al recoger Lanzamiento")
    print(lanzamiento)

    # Idiomas
    lng = info_game(soup,"dd",[{'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max',
                                'data-qa':'gameInfo#releaseInformation#subtitles-value'}])

    idiomas = lng.caracteristica_tipo("Error al recoger Idiomas")
    print(idiomas)

    # Nº de calificaciones

    num_cal = info_game(soup,"span",[{'class':'psw-c-t-2 psw-t-secondary',
                                    'data-qa':'mfe-star-rating#overall-rating#total-ratings'}])

    num_calificaciones = num_cal.caracteristica_tipo("Error al recoger num calificaciones")
    print(num_calificaciones)

    # Calificación PSN
    cal_psn = info_game(soup,"div",[{'class':'psw-t-subtitle psw-t-bold psw-l-line-center','data-qa':'mfe-game-title#average-rating'}])

    calificacion = cal_psn.caracteristica_tipo("Error al recoger Calificación PSN")
    print(calificacion)


    # Calificaciones por estrellas

    cal_stars = info_game(soup,"span",[{'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress1#percentage-label'},
                                {'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress2#percentage-label'},
                                {'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress3#percentage-label'},
                                {'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress4#percentage-label'},
                                {'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress5#percentage-label'}])

    list_stars = cal_stars.caracteristica_tipo("Error al obtener calificaciones por estrella",cal=True)
    calificacion_1 = list_stars[0]
    calificacion_2 = list_stars[1]
    calificacion_3 = list_stars[2]
    calificacion_4 = list_stars[3]
    calificacion_5 = list_stars[4]
    
    print(calificacion_1,calificacion_2,calificacion_3,calificacion_4,calificacion_5)
    
    df_juegos[i].loc[len(df_juegos[i])] = {"id_juego":id_juego,"Titulo":titulo,"Día y hora":fecha_webs,"Plataforma":plataforma,"Genero":genero,"Compañia":compania,"Lanzamiento":lanzamiento,
                                        "Idiomas":idiomas,"Calificación PSN":calificacion,"Número de calificaciones":num_calificaciones,
                                        "Calificación 5 estrellas":calificacion_5,
                                        "Calificación 4 estrellas":calificacion_4,"Calificación 3 estrellas":calificacion_3,
                                        "Calificación 2 estrellas":calificacion_2,"Calificación 1 estrella":calificacion_1,
                                        "Precio original sin PSN":precio_original_sn_psn,"Precio actual sin PSN":precio_actual_sn_psn,"Precio original con PSN":precio_original_cn_psn, "Precio actual con PSN":precio_actual_cn_psn, "País Store":pais_store}
  
df_juegos[0]
df_juegos[1]
df_juegos[2]

/es-es/concept/205354
205354
UNCHARTED: Colección Legado de los Ladrones
2024-03-10T23:51:38.491529
Precio original sin PSN 49,99 €
Precio original con PSN 49,99 €
Precio actual sin PSN 19,99 €
Precio actual con PSN Incluido
PS5
Acción, Aventura
Sony Interactive Entertainment Europe
27/1/2022
Alemán, Checo, Danés, Español, Finlandés, Francés (Francia), Griego, Inglés, Italiano, Neerlandés, Noruego, Polaco, Portugués (Portugal), Ruso, Sueco, Turco
179.894 calificaciones
4.74
3 % 1 % 3 % 8 % 86 %
/en-us/concept/205354
205354
UNCHARTED: Legacy of Thieves Collection
2024-03-10T23:51:42.653717
Precio original sin PSN $49.99
Precio original con PSN $49.99
Precio actual sin PSN $19.99
Precio actual con PSN Included
PS5
Action, Adventure
Sony Interactive Entertainment
1/28/2022
English, French (France), Portuguese (Brazil), Spanish (Mexico)
179,894 ratings
4.74
3% 1% 3% 8% 86%
/ja-jp/concept/205354
205354
アンチャーテッド トレジャーハンターコレクション
2024-03-10T23:51:47.016614
Precio original sin PSN ¥5,390
Precio

,Titulo,Día y hora,Plataforma,Genero,Compañia,Lanzamiento,Idiomas,Calificación PSN,Número de calificaciones,Calificación 5 estrellas,Calificación 4 estrellas,Calificación 3 estrellas,Calificación 2 estrellas,Calificación 1 estrella,Precio original sin PSN,Precio actual sin PSN,Precio original con PSN,Precio actual con PSN,País Store
0,アンチャーテッド トレジャーハンターコレクション,2024-03-10T23:51:47.016614,PS5,"アクション, アドベンチャー",ソニー・インタラクティブエンタテインメント,2022/1/27,"日本語, 英語",4.74,"179,895件の評価",86％,8％,3％,1％,3％,"¥5,390","¥2,156","¥5,390",含まれます,JAP


In [25]:
df_juegos[0]

,Titulo,Día y hora,Plataforma,Genero,Compañia,Lanzamiento,Idiomas,Calificación PSN,Número de calificaciones,Calificación 5 estrellas,Calificación 4 estrellas,Calificación 3 estrellas,Calificación 2 estrellas,Calificación 1 estrella,Precio original sin PSN,Precio actual sin PSN,Precio original con PSN,Precio actual con PSN,País Store
0,Marvel’s Spider-Man 2,2024-03-10T23:11:46.831501,PS5,Acción,Sony Interactive Entertainment Europe,19/10/2023,"Alemán, Checo, Danés, Español, Finlandés, Fran...",4.88,109.089 calificaciones,93 %,4 %,1 %,1 %,1 %,"79,99 €","79,99 €","79,99 €","79,99 €",ESP


In [26]:
df_juegos[1]

,Titulo,Día y hora,Plataforma,Genero,Compañia,Lanzamiento,Idiomas,Calificación PSN,Número de calificaciones,Calificación 5 estrellas,Calificación 4 estrellas,Calificación 3 estrellas,Calificación 2 estrellas,Calificación 1 estrella,Precio original sin PSN,Precio actual sin PSN,Precio original con PSN,Precio actual con PSN,País Store
0,Marvel’s Spider-Man 2,2024-03-10T23:11:50.772913,PS5,Action,Sony Interactive Entertainment,10/20/2023,"English, French (France), Portuguese (Brazil),...",4.88,"109,089 ratings",93%,4%,1%,1%,1%,$69.99,$69.99,$69.99,$69.99,USA


In [27]:
href_valor_list

['/es-es/concept/10002456',
 '/en-us/concept/10002456',
 '/ja-jp/concept/10002456']

Cosas a meter nuevas en variables

In [ ]:
# Lista de recheck
lista_recheck = []


df_juegos_us = df_juegos_es.copy()
df_juegos_jp = df_juegos_es.copy()
df_juegos = [df_juegos_es,df_juegos_us,df_juegos_jp]
lista_store = ["ESP","USA","JAP"]

Lo siguiente en la obtención del href de cada store para después meter un for pasando por cada uno, haciendo una sopa y recopilado la información concreta

In [ ]:
href_store_es = url_game.get('href')

href_store_us = href_store_es.replace('/es-es','/en-us')
href_store_jp = href_store_es.replace('/es-es','/ja-jp')

href_valor_list = [href_store_es,href_store_us,href_store_jp]

In [45]:
soup

<!DOCTYPE html>
<html class="psw-root psw-rem-base" dir="ltr" lang="ja-jp"><head><meta content="2.100.0" name="wca-ssr-version"/><meta content="ja-JP" name="wca-locale"/><meta content="dark" name="wca-theme"/><script id="wca-config-overrides" type="application/json">
    {"client":{"webCheckout":{"clientId":"2eb25762-877f-4140-b341-7c7e14c19f98"}}}</script>
<link crossorigin="" href="https://static.playstation.com" rel="preconnect"/>
<link href="https://static.playstation.com" rel="dns-prefetch"/>
<link href="https://web.np.playstation.com/api/graphql/v1" rel="preconnect"/>
<link href="https://web.np.playstation.com/api/graphql/v1" rel="dns-prefetch"/>
<link crossorigin="" href="https://image.api.playstation.com" rel="preconnect"/>
<link href="https://image.api.playstation.com" rel="dns-prefetch"/>
<link href="https://id.sonyentertainmentnetwork.com" rel="dns-prefetch"/>
<link href="https://checkout.playstation.com" rel="dns-prefetch"/>
<link href="https://static.playstation.com/font-s

buscando mejores formas de extraer datos con diccionarios desde un solo boton

In [46]:
dict_completo = soup.find('button',attrs={'data-qa':'mfeCtaMain#cta#action'}).get_attribute_list('data-telemetry-meta')[0]

In [47]:
import json
conv_json = json.loads(dict_completo)

Titulo  
Día y hora  
Plataforma  
Genero  
Compañia  
Lanzamiento  
Idiomas  
Calificación PSN  
Número de calificaciones  
Calificación 5 estrellas  
Calificación 4 estrellas  
Calificación 3 estrellas  
Calificación 2 estrellas  
Calificación 1 estrella  
Precio original sin PSN	  
Precio actual sin PSN  
Precio original con PSN  
Precio actual con PSN  
País Store

In [48]:
conv_json

{'conceptId': '205354',
 'ctaIndex': 0,
 'ctaSubType': 'add_to_cart',
 'ctaType': 'ADD_TO_CART',
 'interactAction': 'click add to cart',
 'interactCta': 'add to cart',
 'productId': 'JP9000-PPSA05685_00-UNCHARTED4R00000',
 'productDetail': [{'productId': 'JP9000-PPSA05685_00-UNCHARTED4R00000',
   'productName': 'アンチャーテッド トレジャーハンターコレクション',
   'productToSkuIds': ['JP9000-PPSA05685_00-UNCHARTED4R00000-J002',
    'JP9000-PPSA05685_00-UNCHARTED4R00000-J003'],
   'productPriceDetail': [{'rewardId': 'OFFER-PROD-7636_60',
     'offerCampaignId': 'PROMO-PROD-00007636',
     'originalPriceFormatted': '¥5,390',
     'originalPriceValue': 5390,
     'discountPriceFormatted': '¥2,156',
     'discountPriceValue': 2156,
     'priceCurrencyCode': 'JPY',
     'offerBranding': 'NONE',
     'offerQualification': [],
     'offerExclusive': False,
     'offerApplied': True,
     'offerApplicability': 'APPLICABLE',
     'offerIsTiedToSubscription': False},
    {'rewardId': 'SUB001-SUBC00002_00-HM93SKR5MGPG6

Poner el tipo de descuento y porque, no solo plataforma PSN

In [49]:
conv_json['conceptId'] # ID

'205354'

In [50]:
conv_json['productId']

'JP9000-PPSA05685_00-UNCHARTED4R00000'

In [82]:
conv_json['productDetail'][0]

{'productId': 'JP9000-PPSA05685_00-UNCHARTED4R00000',
 'productName': 'アンチャーテッド トレジャーハンターコレクション',
 'productToSkuIds': ['JP9000-PPSA05685_00-UNCHARTED4R00000-J002',
  'JP9000-PPSA05685_00-UNCHARTED4R00000-J003'],
 'productPriceDetail': [{'rewardId': 'OFFER-PROD-7636_60',
   'offerCampaignId': 'PROMO-PROD-00007636',
   'originalPriceFormatted': '¥5,390',
   'originalPriceValue': 5390,
   'discountPriceFormatted': '¥2,156',
   'discountPriceValue': 2156,
   'priceCurrencyCode': 'JPY',
   'offerBranding': 'NONE',
   'offerQualification': [],
   'offerExclusive': False,
   'offerApplied': True,
   'offerApplicability': 'APPLICABLE',
   'offerIsTiedToSubscription': False},
  {'rewardId': 'SUB001-SUBC00002_00-HM93SKR5MGPG6TD8',
   'offerCampaignId': None,
   'originalPriceFormatted': '¥5,390',
   'originalPriceValue': 5390,
   'discountPriceFormatted': '含まれます',
   'discountPriceValue': 0,
   'priceCurrencyCode': 'JPY',
   'offerBranding': 'PS_PLUS',
   'offerQualification': [{'__typename': 'Q

In [83]:
conv_json['productDetail'][0]['productPriceDetail'][1]['originalPriceFormatted'] # Precio original

'¥5,390'

In [67]:
conv_json['productDetail'][0]['productPriceDetail'][1]['discountPriceValue'] # Precio actual con PSN u otro servicio

0

In [75]:
conv_json['productDetail'][0]['productPriceDetail'][1]['offerBranding'] # Motivo de oferta, campaña

'PS_PLUS'

In [70]:
conv_json['productDetail'][0]['productPriceDetail'][0]['originalPriceFormatted'] # Precio original sin PSN

'¥5,390'

In [73]:
conv_json['productDetail'][0]['productPriceDetail'][0]['discountPriceFormatted'] # Precio actual sin PSN

'¥2,156'

In [ ]:
conv_json['productDetail']

[{'productId': 'JP9000-PPSA05685_00-UNCHARTED4R00000',
  'productName': 'アンチャーテッド トレジャーハンターコレクション',
  'productToSkuIds': ['JP9000-PPSA05685_00-UNCHARTED4R00000-J002',
   'JP9000-PPSA05685_00-UNCHARTED4R00000-J003'],
  'productPriceDetail': [{'rewardId': 'OFFER-PROD-7636_60',
    'offerCampaignId': 'PROMO-PROD-00007636',
    'originalPriceFormatted': '¥5,390',
    'originalPriceValue': 5390,
    'discountPriceFormatted': '¥2,156',
    'discountPriceValue': 2156,
    'priceCurrencyCode': 'JPY',
    'offerBranding': 'NONE',
    'offerQualification': [],
    'offerExclusive': False,
    'offerApplied': True,
    'offerApplicability': 'APPLICABLE',
    'offerIsTiedToSubscription': False},
   {'rewardId': 'SUB001-SUBC00002_00-HM93SKR5MGPG6TD8',
    'offerCampaignId': None,
    'originalPriceFormatted': '¥5,390',
    'originalPriceValue': 5390,
    'discountPriceFormatted': '含まれます',
    'discountPriceValue': 0,
    'priceCurrencyCode': 'JPY',
    'offerBranding': 'PS_PLUS',
    'offerQualific

In [ ]:
conv_json['productDetail']

[{'productId': 'JP9000-PPSA05685_00-UNCHARTED4R00000',
  'productName': 'アンチャーテッド トレジャーハンターコレクション',
  'productToSkuIds': ['JP9000-PPSA05685_00-UNCHARTED4R00000-J002',
   'JP9000-PPSA05685_00-UNCHARTED4R00000-J003'],
  'productPriceDetail': [{'rewardId': 'OFFER-PROD-7636_60',
    'offerCampaignId': 'PROMO-PROD-00007636',
    'originalPriceFormatted': '¥5,390',
    'originalPriceValue': 5390,
    'discountPriceFormatted': '¥2,156',
    'discountPriceValue': 2156,
    'priceCurrencyCode': 'JPY',
    'offerBranding': 'NONE',
    'offerQualification': [],
    'offerExclusive': False,
    'offerApplied': True,
    'offerApplicability': 'APPLICABLE',
    'offerIsTiedToSubscription': False},
   {'rewardId': 'SUB001-SUBC00002_00-HM93SKR5MGPG6TD8',
    'offerCampaignId': None,
    'originalPriceFormatted': '¥5,390',
    'originalPriceValue': 5390,
    'discountPriceFormatted': '含まれます',
    'discountPriceValue': 0,
    'priceCurrencyCode': 'JPY',
    'offerBranding': 'PS_PLUS',
    'offerQualific

In [ ]:
conv_json['productDetail']

[{'productId': 'JP9000-PPSA05685_00-UNCHARTED4R00000',
  'productName': 'アンチャーテッド トレジャーハンターコレクション',
  'productToSkuIds': ['JP9000-PPSA05685_00-UNCHARTED4R00000-J002',
   'JP9000-PPSA05685_00-UNCHARTED4R00000-J003'],
  'productPriceDetail': [{'rewardId': 'OFFER-PROD-7636_60',
    'offerCampaignId': 'PROMO-PROD-00007636',
    'originalPriceFormatted': '¥5,390',
    'originalPriceValue': 5390,
    'discountPriceFormatted': '¥2,156',
    'discountPriceValue': 2156,
    'priceCurrencyCode': 'JPY',
    'offerBranding': 'NONE',
    'offerQualification': [],
    'offerExclusive': False,
    'offerApplied': True,
    'offerApplicability': 'APPLICABLE',
    'offerIsTiedToSubscription': False},
   {'rewardId': 'SUB001-SUBC00002_00-HM93SKR5MGPG6TD8',
    'offerCampaignId': None,
    'originalPriceFormatted': '¥5,390',
    'originalPriceValue': 5390,
    'discountPriceFormatted': '含まれます',
    'discountPriceValue': 0,
    'priceCurrencyCode': 'JPY',
    'offerBranding': 'PS_PLUS',
    'offerQualific

codigo EA offer

<button data-qa="mfeCtaMain#cta#action" data-track="{&quot;eventInfo&quot;:{&quot;eventAction&quot;:&quot;click&quot;,&quot;eventName&quot;:&quot;add to cart&quot;},&quot;attributes&quot;:{&quot;moduleName&quot;:&quot;pricing cta&quot;,&quot;assetName&quot;:&quot;&quot;,&quot;label&quot;:&quot;add to cart&quot;,&quot;position&quot;:&quot;1&quot;,&quot;linkUrl&quot;:&quot;&quot;,&quot;elementType&quot;:&quot;button&quot;,&quot;userActionFlag&quot;:false,&quot;sku&quot;:&quot;EP0006-PPSA03846_00-BEYONDPS5GAME000-E003&quot;}}" data-track-click="ctaWithPrice:addToCart" data-telemetry-meta="{&quot;conceptId&quot;:&quot;10002958&quot;,&quot;ctaIndex&quot;:0,&quot;ctaSubType&quot;:&quot;add_to_cart&quot;,&quot;ctaType&quot;:&quot;ADD_TO_CART&quot;,&quot;interactAction&quot;:&quot;click add to cart&quot;,&quot;interactCta&quot;:&quot;add to cart&quot;,&quot;productId&quot;:&quot;EP0006-PPSA03846_00-BEYONDPS5GAME000&quot;,&quot;productDetail&quot;:[{&quot;productId&quot;:&quot;EP0006-PPSA03846_00-BEYONDPS5GAME000&quot;,&quot;productName&quot;:&quot;Dead Space&quot;,&quot;productToSkuIds&quot;:[&quot;EP0006-PPSA03846_00-BEYONDPS5GAME000-E003&quot;],&quot;productPriceDetail&quot;:[{&quot;rewardId&quot;:&quot;&quot;,&quot;offerCampaignId&quot;:null,&quot;originalPriceFormatted&quot;:&quot;79,99&nbsp;€&quot;,&quot;originalPriceValue&quot;:7999,&quot;discountPriceFormatted&quot;:&quot;79,99&nbsp;€&quot;,&quot;discountPriceValue&quot;:7999,&quot;priceCurrencyCode&quot;:&quot;EUR&quot;,&quot;offerBranding&quot;:&quot;NONE&quot;,&quot;offerQualification&quot;:[],&quot;offerExclusive&quot;:false,&quot;offerApplied&quot;:false,&quot;offerApplicability&quot;:&quot;APPLICABLE&quot;,&quot;offerIsTiedToSubscription&quot;:false},{&quot;rewardId&quot;:&quot;EP5679-CUSA15082_00-RWD-8571459&quot;,&quot;offerCampaignId&quot;:null,&quot;originalPriceFormatted&quot;:&quot;79,99&nbsp;€&quot;,&quot;originalPriceValue&quot;:7999,&quot;discountPriceFormatted&quot;:&quot;Incluido&quot;,&quot;discountPriceValue&quot;:0,&quot;priceCurrencyCode&quot;:&quot;EUR&quot;,&quot;offerBranding&quot;:&quot;EA_ACCESS&quot;,&quot;offerQualification&quot;:[{&quot;__typename&quot;:&quot;Qualification&quot;,&quot;type&quot;:&quot;ENTITLEMENT_IN_CART&quot;,&quot;value&quot;:&quot;EP5679-CUSA15082_00-RWD-8571459&quot;}],&quot;offerExclusive&quot;:false,&quot;offerApplied&quot;:false,&quot;offerApplicability&quot;:&quot;UPSELL&quot;,&quot;offerIsTiedToSubscription&quot;:true}]}],&quot;skuDetail&quot;:[{&quot;skuId&quot;:&quot;EP0006-PPSA03846_00-BEYONDPS5GAME000-E003&quot;,&quot;skuPriceDetail&quot;:[{&quot;rewardId&quot;:&quot;&quot;,&quot;offerCampaignId&quot;:null,&quot;originalPriceFormatted&quot;:&quot;79,99&nbsp;€&quot;,&quot;originalPriceValue&quot;:7999,&quot;discountPriceFormatted&quot;:&quot;79,99&nbsp;€&quot;,&quot;discountPriceValue&quot;:7999,&quot;priceCurrencyCode&quot;:&quot;EUR&quot;,&quot;offerBranding&quot;:&quot;NONE&quot;,&quot;offerQualification&quot;:[],&quot;offerExclusive&quot;:false,&quot;offerApplied&quot;:false,&quot;offerApplicability&quot;:&quot;APPLICABLE&quot;,&quot;offerIsTiedToSubscription&quot;:false}],&quot;skuName&quot;:&quot;Juego&quot;}],&quot;titleId&quot;:&quot;PPSA03846_00&quot;}" id="" class="psw-fill-x dtm-track psw-button psw-b-0 psw-t-button psw-l-line-center psw-button-sizing psw-button-sizing--medium psw-purchase-button psw-solid-button" tabindex="0" role="" type="button" name=""><span class="psw-fill-x psw-t-truncate-1 psw-l-space-x-2 ">Añadir al carrito</span></button>

codigo psn offer

<button data-qa="mfeCtaMain#cta#action" data-track="{&quot;eventInfo&quot;:{&quot;eventAction&quot;:&quot;click&quot;,&quot;eventName&quot;:&quot;add to cart&quot;},&quot;attributes&quot;:{&quot;moduleName&quot;:&quot;pricing cta&quot;,&quot;assetName&quot;:&quot;&quot;,&quot;label&quot;:&quot;add to cart&quot;,&quot;position&quot;:&quot;1&quot;,&quot;linkUrl&quot;:&quot;&quot;,&quot;elementType&quot;:&quot;button&quot;,&quot;userActionFlag&quot;:false,&quot;sku&quot;:&quot;EP9000-PPSA05684_00-UNCHARTED4R00000-E003&quot;}}" data-track-click="ctaWithPrice:addToCart" data-telemetry-meta="{&quot;conceptId&quot;:&quot;205354&quot;,&quot;ctaIndex&quot;:0,&quot;ctaSubType&quot;:&quot;add_to_cart&quot;,&quot;ctaType&quot;:&quot;ADD_TO_CART&quot;,&quot;interactAction&quot;:&quot;click add to cart&quot;,&quot;interactCta&quot;:&quot;add to cart&quot;,&quot;productId&quot;:&quot;EP9000-PPSA05684_00-UNCHARTED4R00000&quot;,&quot;productDetail&quot;:[{&quot;productId&quot;:&quot;EP9000-PPSA05684_00-UNCHARTED4R00000&quot;,&quot;productName&quot;:&quot;UNCHARTED: Colección Legado de los Ladrones&quot;,&quot;productToSkuIds&quot;:[&quot;EP9000-PPSA05684_00-UNCHARTED4R00000-E003&quot;],&quot;productPriceDetail&quot;:[{&quot;rewardId&quot;:&quot;OFFER-PROD-7634_60&quot;,&quot;offerCampaignId&quot;:&quot;PROMO-PROD-00007634&quot;,&quot;originalPriceFormatted&quot;:&quot;49,99&nbsp;€&quot;,&quot;originalPriceValue&quot;:4999,&quot;discountPriceFormatted&quot;:&quot;19,99&nbsp;€&quot;,&quot;discountPriceValue&quot;:1999,&quot;priceCurrencyCode&quot;:&quot;EUR&quot;,&quot;offerBranding&quot;:&quot;NONE&quot;,&quot;offerQualification&quot;:[],&quot;offerExclusive&quot;:false,&quot;offerApplied&quot;:true,&quot;offerApplicability&quot;:&quot;APPLICABLE&quot;,&quot;offerIsTiedToSubscription&quot;:false},{&quot;rewardId&quot;:&quot;SUB001-SUBC00002_00-TJ4KZ94GWFC3NYB0&quot;,&quot;offerCampaignId&quot;:null,&quot;originalPriceFormatted&quot;:&quot;49,99&nbsp;€&quot;,&quot;originalPriceValue&quot;:4999,&quot;discountPriceFormatted&quot;:&quot;Incluido&quot;,&quot;discountPriceValue&quot;:0,&quot;priceCurrencyCode&quot;:&quot;EUR&quot;,&quot;offerBranding&quot;:&quot;PS_PLUS&quot;,&quot;offerQualification&quot;:[{&quot;__typename&quot;:&quot;Qualification&quot;,&quot;type&quot;:&quot;ENTITLEMENT_IN_CART&quot;,&quot;value&quot;:&quot;SUB001-SUBC00002_00-TJ4KZ94GWFC3NYB0&quot;}],&quot;offerExclusive&quot;:false,&quot;offerApplied&quot;:false,&quot;offerApplicability&quot;:&quot;UPSELL&quot;,&quot;offerIsTiedToSubscription&quot;:true}]}],&quot;skuDetail&quot;:[{&quot;skuId&quot;:&quot;EP9000-PPSA05684_00-UNCHARTED4R00000-E003&quot;,&quot;skuPriceDetail&quot;:[{&quot;rewardId&quot;:&quot;OFFER-PROD-7634_60&quot;,&quot;offerCampaignId&quot;:&quot;PROMO-PROD-00007634&quot;,&quot;originalPriceFormatted&quot;:&quot;49,99&nbsp;€&quot;,&quot;originalPriceValue&quot;:4999,&quot;discountPriceFormatted&quot;:&quot;19,99&nbsp;€&quot;,&quot;discountPriceValue&quot;:1999,&quot;priceCurrencyCode&quot;:&quot;EUR&quot;,&quot;offerBranding&quot;:&quot;NONE&quot;,&quot;offerQualification&quot;:[],&quot;offerExclusive&quot;:false,&quot;offerApplied&quot;:true,&quot;offerApplicability&quot;:&quot;APPLICABLE&quot;,&quot;offerIsTiedToSubscription&quot;:false}],&quot;skuName&quot;:&quot;Juego&quot;}],&quot;titleId&quot;:&quot;PPSA05684_00&quot;}" id="" class="psw-fill-x dtm-track psw-button psw-b-0 psw-t-button psw-l-line-center psw-button-sizing psw-button-sizing--medium psw-purchase-button psw-solid-button" tabindex="0" role="" type="button" name=""><span class="psw-fill-x psw-t-truncate-1 psw-l-space-x-2 ">Añadir al carrito</span></button>